## Coursera IBM Capstone Project

#### Week 4 
##### A description of the problem and a discussion of the background
##### A description of the data and how it will be used to solve the problem

### Introduction
#### Background
##### *Toronto* is Toronto is the capital city of the Canadian province of Ontario with population 2.7 Million. It is the most populous city in Canada and the fourth most populous city in North America.People have travelled through and inhabited the Toronto area, located on a broad sloping plateau interspersed with rivers, deep ravines, and urban forest, for more than 10,000 years. Now, Toronto is an international centre of business, finance, arts, and culture, and is recognized as one of the most multicultural and cosmopolitan cities in the world. Some neighbourhoods are Old Toronto, Downtown Core (Central), East End, North End, West End, East York, Etobicoke, North York, Scarborough, York.

##### *NYC* is New York City is often called just New York is the most populous city in the United States. It's population is 8.3 Million. Just like toronoto it is filled with people of different nationalities and it is a multicultural city. The five boroughs—Brooklyn, Queens, Manhattan, the Bronx, and Staten Island—were consolidated into a single city in 1898. Some neighbourhoods are Central Bronx, Central Brooklyn, Borough Park, Flatbush, Greenpoint, East Harlem, Upper West Side, Jamaica, Rockaways, West Queens.

#### Problem Statement
##### Toronto and NYC are top cities for tourism in the world. The tourism characteristics should be compared in this project. This study or project should be able to use restruant, museums, gallaries for comparision.  
##### Thus this project should be able to help business people who are trying to select a good neighbourhood for open restuarants. 

#### Scope of Study
##### In case of Toronto, the entire city area is taken for study. For NYC, some neighbourhoods in the study are Central Bronx, Central Brooklyn, Borough Park, Flatbush, Greenpoint, East Harlem, Upper West Side, Jamaica, Rockaways, West Queens. 

#### Data
##### The number of restaurants, museums, galleries in Toronto and NYC are compared. The data sources that explain the postal codes and corresponding neighborhoods/ boroughs/ districts of the two locations of interest are extracted from 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' (Toronto, Ontario). The postal codes for neighbourhoods in NYC are obtained via 'https://www.health.ny.gov/statistics/cancer/registry/appendix/neighborhoods.htm' 
##### The coordinates of the postal codes based on the district/ borough/ neoghbourhood of both locations are extracted via 'http://cocl.us/Geospatial_data' (Toronto, Ontario).
##### The geospatial data, numbers and other detials of the restaurants, museums and galleries of both location of interest are extracted via Foursquare API.



In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


In [30]:
!pip install lxml
!pip install lxml html5lib beautifulsoup4
!pip install geocoder
!pip install folium
!pip install geopy
from geopy.geocoders import Nominatim 

In [4]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))

In [12]:
PostData = df[0] 

PostData=PostData[PostData.Borough != 'Not assigned'].reset_index() #Cleaning the not assigned borough names
PostData = PostData.drop(['index'], axis=1)
PostData.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [13]:
print(PostData.shape)

(103, 3)


In [14]:
url = 'https://cocl.us/Geospatial_data'
Geo = pd.read_csv(url,sep=',')
Lats=[]
Longs=[]
for i, row in PostData.iterrows():
    for j, row in Geo.iterrows():
        if PostData.loc[i,"Postal Code"]==Geo.loc[j,"Postal Code"]:
            Lats.append(Geo.loc[j,"Latitude"])
            Longs.append(Geo.loc[j,"Longitude"])
PostData['Lat'] = Lats
PostData['Long'] = Longs
PostData.head()

,Postal Code,Borough,Neighbourhood,Lat,Long
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [25]:
import json # Handling JSON files
import requests # Library to handle requests
from pandas.io.json import json_normalize # Transform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium as fl # Map Rendering as

In [26]:
# Inputing Toronto's coordinates
lat=43.651070
long = -79.347015
#Creating city map
map_toronto = fl.Map(location=[lat,long],zoom_start = 10)
for index, rows in PostData.iterrows():
    lat = PostData.loc[index,"Lat"]
    long = PostData.loc[index,"Long"]
    Neigh = PostData.loc[index,"Neighbourhood"]
    Borough = PostData.loc[index,"Borough"]
    label = '{},{}'.format(Neigh,Borough)
    label = fl.Popup(label,parse_html=True)
    fl.CircleMarker(
    [lat,long],
    radius=10,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
#Well, here we iterate from all the neibhbourhoods in data and add then to our map.
map_toronto


In [28]:
nyc = [[['Central Bronx'],[10453]],[['Central Brooklyn'],[11212]],[['Borough Park'],[11204]],[['Flatbush'],[11203]], \
                              [['Greenpoint'],[11211]],[['East Harlem'],[10029]], \
                              [['Upper West Side'],[10023]],[['Jamaica'],[11412]],[['Rockaways'],[11691]],[['West Queens'],[11368]]]

district_list =[]
postal_code_list=[]

for dist in nyc:
    for dist_name in dist[0]:
        for post_code in dist[1]:
            district_list.append(dist_name)
            postal_code_list.append(post_code)
     
    nyc_df=pd.DataFrame({'District':district_list,'Postal Code':postal_code_list}) #To create Dataframe with districts and postal code information

nyc_df #Displaying the resulted dataframe

,District,Postal Code
0,Central Bronx,10453
1,Central Brooklyn,11212
2,Borough Park,11204
3,Flatbush,11203
4,Greenpoint,11211
5,East Harlem,10029
6,Upper West Side,10023
7,Jamaica,11412
8,Rockaways,11691
9,West Queens,11368


In [31]:
geolocator = Nominatim(user_agent="foursquare_agent")

lat_list =[] #To create a list to store latitude values
lng_list=[] #To create a list to store longitude values

#To obtain geospatial data for the corresponding districts and postal codes
for pc, d in zip(nyc_df['Postal Code'],nyc_df['District']):
   
    location = geolocator.geocode('{},{}'.format(pc,d))
    latitude = location.latitude
    longitude = location.longitude
    lat_list.append(latitude)
    lng_list.append(longitude)

nyc_df['Latitude']=lat_list
nyc_df['Longitude']=lng_list

nyc_df # To display dataframe with geospatial data

,District,Postal Code,Latitude,Longitude
0,Central Bronx,10453,40.850656,-73.866524
1,Central Brooklyn,11212,40.650104,-73.949582
2,Borough Park,11204,40.633993,-73.996806
3,Flatbush,11203,40.652048,-73.959027
4,Greenpoint,11211,40.723713,-73.950971
5,East Harlem,10029,40.794722,-73.942500
6,Upper West Side,10023,40.787045,-73.975416
7,Jamaica,11412,40.691485,-73.805677
8,Rockaways,11691,-33.908704,18.398713
9,West Queens,11368,40.749824,-73.797634


In [34]:
map_nyc = folium.Map(location=[40.7896239, -73.9598939], zoom_start=13)

# add markers to map
for lat, lng, district, postalcode in zip(nyc_df['Latitude'], nyc_df['Longitude'], nyc_df['District'], nyc_df['Postal Code']):
    label = '{}, {}'.format(postalcode, district)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_nyc)  
    
map_nyc

In [35]:
CLIENT_ID = 'ZZ15HRJJZMST2QNGKC4OM2FXSICLGMRCJYPS21NWC5Q4R04F' # your Foursquare ID
CLIENT_SECRET = 'ZKPBWAX12PWFB1SH2THCWKXU43YUNJ3HRBIJVQ2LKRMWOKH5' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 500
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZZ15HRJJZMST2QNGKC4OM2FXSICLGMRCJYPS21NWC5Q4R04F
CLIENT_SECRET:ZKPBWAX12PWFB1SH2THCWKXU43YUNJ3HRBIJVQ2LKRMWOKH5
